# Model

In [ ]:
# Model source
import sys
sys.path.append("/home/homer/ekiapp/finn/notebooks/GitHub/M_Project/Patitioning/tmp")
sys.path.append("/home/homer/ekiapp/finn/deps/brevitas/src/brevitas_examples")
from bnn_pynq.models.CNV import*

# Configuration
import configparser
config = configparser.ConfigParser()
config['QUANT'] = {'WEIGHT_BIT_WIDTH': '1',
                     'ACT_BIT_WIDTH': '1',
                     'IN_BIT_WIDTH': '8'}
config['MODEL'] = {'NUM_CLASSES':'2',
                  'IN_CHANNELS':'3'}
# Model
model = cnv(config)

In [ ]:
import torch
from qonnx.core.modelwrapper import ModelWrapper

# trained model
model_ref = model
model_ref.load_state_dict(torch.load('../../../../notebooks/GitHub/M_Project/Model/tmp/cnv_e50_1bit_trained.pth'))

# transformed model
model_ut = ModelWrapper("../../../../notebooks/GitHub/M_Project/Patitioning/tmp/cnv_1bit_trained_finn.onnx_streamlined.onnx")

# Dataset

In [ ]:
# Dependencies
import numpy as np
import finn.core.onnx_exec as oxe
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Compose
from torchvision.models import *
import random
import gc 
import os
torch.manual_seed(0)

# configure device
if (torch.cuda.is_available()):
    dev = 'cuda'
else:
    dev = 'cpu'
device = dev
if (torch.cuda.is_available()):
    torch.cuda.device(device)
    gc.collect()
    torch.cuda.empty_cache()
print("using " + device)

# dataset
path_dataset = "/home/homer/ekiapp/finn/notebooks/dataset/SugarWeed"#"/home/homer/Downloads/dataset/SugarWeed"
batch_size = 16
train_set = 0.8
test_set = 0.2
img_size = 32
rand_rotation = 30

# Create data loaders.
data_dir = path_dataset

# Applying Transformation
train_transforms = transforms.Compose([
                                transforms.RandomRotation(rand_rotation),
                                transforms.RandomResizedCrop(img_size),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor()])
test_dataset = datasets.ImageFolder(f"{data_dir}/val", transform=train_transforms)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

# Validation

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images[0]
        images = images.numpy() *255
        images_np = images#.astype(np.uint8)
        images = torch.from_numpy(images_np.reshape(1,3,32,32))
        
        # model ref
        input_brevitas = images
        outputs_ref = model_ref(input_brevitas)
        # model_ut
        input_dict = {"global_in": images_np.reshape(1,3,32,32)}
        output_dict = oxe.execute_onnx(model_ut, input_dict, return_full_exec_context=False)
        output_ut = output_dict[list(output_dict.keys())[0]]

        # compare
        try:
            assert np.isclose(output_ut.flatten()[0], torch.argmax(outputs_ref).item(), atol=1e-3).all()
            print("Results are the same!")
        except AssertionError:
            print ("output_ut.flatten()[0]:\t" + str(output_ut.flatten()[0]))
            print ("torch.argmax(outputs_ref).item():\t" + str(torch.argmax(outputs_ref).item()))
            print ("The results are not the same!")
            #assert False, "The results are not the same!"




# Accuracy check

In [ ]:
def accuracy_check(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.numpy() *255
            images = images.astype(np.uint8)
            images = torch.from_numpy(images)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

model.load_state_dict(torch.load('/home/homer/ekiapp/finn/notebooks/GitHub/M_Project/Model/tmp/cnv_e50_1bit_trained.pth'))#cnv_e50_1bit_trained.pth'))#cnv_2epoch_1bit_adam_trained.pth'))
torch.manual_seed(0)
accuracy = accuracy_check(model, testloader)

In [ ]:
accuracy

In [ ]:
first_batch = next(iter(testloader)) #list
print(first_batch)

In [ ]:
data_np = first_batch[0].numpy()
labels_np = first_batch[1].numpy()

In [ ]:
data_np.shape[0]

In [ ]:
labels_np.flatten()[0]